In [3]:
from glob import glob
import pyarrow.parquet as pq
import pandas as pd
ROOT_DIR = "/data/data/fintuning/infinity-instruct/7M"
all_files = glob(ROOT_DIR+"/*")
print(len(all_files), all_files)
from tqdm import tqdm
import json
from transformers import AutoTokenizer
model_path = "/data/model/llm/hf_model/steel-llm-step-1060000-ckpt"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
import numpy as np

75 ['/data/data/fintuning/infinity-instruct/7M/train-00013-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00005-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00016-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00014-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00011-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00022-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00017-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00052-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00001-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00071-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00072-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00033-of-00075.parquet', '/data/data/fintuning/infinity-instruct/7M/train-00000-of-00075.parquet', '/data/data/fintuning/infinity-ins

# 转为llama factory需要的格式

In [5]:

def to_alpaca_data(str_json_data):
    return {
                "instruction": str_json_data[-2]["value"],
                "input": "",
                "output": str_json_data[-1]["value"]
            }

def to_share_gpt_data(str_json_data):
    conversations = str_json_data.tolist()
    text = ""
    for item in conversations:
        text += item["value"]
    return {
        "conversations": conversations
    }, text

def process_one_data(file_dir):
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    # Index(['id', 'conversations', 'label', 'langdetect', 'source'], dtype='object')
    # print(df.columns)
    # 可以过滤=======================
    # need_idx = df["langdetect"] == "zh-cn"
    # df = df.loc[need_idx, :]
    #===============================
    str_json_datas = df["conversations"]
    language_datas = df["langdetect"]
    muti_turn_counter = 0
    odd_cov_counter = 0
    encounter = 0
    a_file_datas = []
    text_list = []
    for idx, str_json_data in  enumerate(str_json_datas):
        L = len(str_json_data)
        # 过滤中文
        if language_datas.iloc[idx] != "zh-cn":
            encounter += 1
            continue
        if L > 2:
            muti_turn_counter += 1
        if L % 2 == 1:
            odd_cov_counter += 1
        # a_file_datas.append(to_alpaca_data(str_json_data)) 
        conversation, text= to_share_gpt_data(str_json_data)
        text_list.append(text)
        a_file_datas.append(conversation)
    
    input_ids = tokenizer(text_list)['input_ids']
    token_num_list = [len(x) for x in input_ids]
    print(f"all data:{len(str_json_datas)} muti_turn_counter:{muti_turn_counter} odd_cov_counter:{odd_cov_counter} encounter:{encounter}")
    return a_file_datas,token_num_list

all_datas = []
token_nums = []
for file_dir in tqdm(all_files):
    datas,token_num_list = process_one_data(file_dir) 
    all_datas.extend(datas) 
    token_nums.extend(token_num_list)
print(len(all_datas))
print(len(token_nums))

  1%|▏         | 1/75 [00:02<02:37,  2.13s/it]

all data:100000 muti_turn_counter:2410 odd_cov_counter:0 encounter:89901


  3%|▎         | 2/75 [00:04<02:42,  2.23s/it]

all data:100000 muti_turn_counter:2400 odd_cov_counter:0 encounter:90014


  4%|▍         | 3/75 [00:06<02:41,  2.24s/it]

...

all data:100000 muti_turn_counter:2463 odd_cov_counter:0 encounter:89824


 99%|█████████▊| 74/75 [02:25<00:01,  1.89s/it]

all data:100000 muti_turn_counter:2439 odd_cov_counter:0 encounter:90171


100%|██████████| 75/75 [02:27<00:00,  1.97s/it]

all data:100000 muti_turn_counter:2473 odd_cov_counter:0 encounter:89826
751313
751313


# 统计分位数，过滤过长的样本

In [13]:
import numpy as np
np.percentile(token_nums,[50, 60, 70, 80, 90,95,99])
print(np.percentile(token_nums,[50, 60, 70, 80, 90,95, 99]))
filtered_long_data = []
filtered_token_num = []
for i in range(len(all_datas)):
    if token_nums[i] < 768:
        filtered_long_data.append(all_datas[i])
        filtered_token_num.append(token_nums[i])
print(np.percentile(filtered_token_num,[50, 60, 70, 80, 90,95, 99]))
print(len(filtered_token_num))

[ 252.  281.  317.  368.  474.  666. 1692.]
[247. 274. 306. 349. 423. 504. 679.]
722236


In [14]:
json_dir = "/data/data/fintuning/llamafactory_input/infinity_chinese_data.jsonl"
with open(json_dir, 'w', encoding="utf-8") as f:
    for item in filtered_long_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [12]:
# 随机保存7w个数据
import random
random_data = random.sample(all_datas, 100000)
print(type(all_datas[0]))
import json
print(all_datas[0])
json_dir = "/data/data/fintuning/infinity-instruct/all_data_10w.json"
with open(json_dir, "w") as file:
    json.dump(all_datas, file)

<class 'dict'>
{'conversations': [{'from': 'human', 'value': 'A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around US\\$90, its Japanese inventors said Tuesday.\n\nWhat best summarizes the content of the above article?\npick from the following. (I). World. (II). Sports. (III). Business. (IV). Science/Tech.'}, {'from': 'gpt', 'value': '(IV).'}]}


In [3]:
import json
with open("/data/data/fintuning/infinity-instruct/7M/all_data_7M_v1.json", "r") as file:
    read_data = json.load(file)